In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import dmc
from process import processed_data

Tensorflow not installed


In [3]:
data = processed_data()
train_ids, test_ids = dmc.loading.load_ids('rawMirrored')
train, test = dmc.preprocessing.split_train_test(data, train_ids, test_ids)

## Neural Network evaluation using classic ensemble splits

In [4]:
# allocate memory
data = train_ids = test_ids = None
####
ensemble = dmc.ensemble.Ensemble(train, test)

In [5]:
for split in ensemble.splits:
    print(len(ensemble.splits[split][1]), split)

2082 unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup
14 unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup
22160 unknown_articleID-unknown_customerID-known_voucherID-known_productGroup
3239 unknown_articleID-known_customerID-unknown_voucherID-known_productGroup
12 unknown_articleID-known_customerID-known_voucherID-unknown_productGroup
62265 unknown_articleID-known_customerID-known_voucherID-known_productGroup
4189 known_articleID-unknown_customerID-unknown_voucherID-known_productGroup
42844 known_articleID-unknown_customerID-known_voucherID-known_productGroup
6496 known_articleID-known_customerID-unknown_voucherID-known_productGroup
93197 known_articleID-known_customerID-known_voucherID-known_productGroup


Print splits and names to know where to not drop customer for naive bayes.

### Baseline
```
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup : size  4189  prec  0.659823346860826
--------------------------------------
known_articleID-known_customerID-known_voucherID-known_productGroup : size  93197  prec  0.6745603399251049
--------------------------------------
unknown_articleID-known_customerID-known_voucherID-known_productGroup : size  62265  prec  0.6728338552959126
--------------------------------------
unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup : size  14  prec  0.8571428571428572
--------------------------------------
known_articleID-known_customerID-unknown_voucherID-known_productGroup : size  6496  prec  0.6657943349753694
--------------------------------------
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup : size  22160  prec  0.6546931407942238
--------------------------------------
unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup : size  2082  prec  0.6652257444764649
--------------------------------------
unknown_articleID-known_customerID-known_voucherID-unknown_productGroup : size  12  prec  0.6666666666666667
--------------------------------------
known_articleID-unknown_customerID-known_voucherID-known_productGroup : size  42844  prec  0.6607459620950424
--------------------------------------
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup : size  3239  prec  0.6687249150972523
--------------------------------------
Overall : 0.6690881106816972 78260
```


### SVM with 50k sample
```

unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6714697406340058 2082
unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup 0.8571428571428572 14
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6607851985559567 22160
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6644025933930225 3239
unknown_articleID-known_customerID-known_voucherID-unknown_productGroup 0.6666666666666667 12
unknown_articleID-known_customerID-known_voucherID-known_productGroup 0.6448727214325864 62265
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6719980902363333 4189
known_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6569881430305293 42844
known_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6366995073891626 6496
known_articleID-known_customerID-known_voucherID-known_productGroup 0.6434649183986609 93197
```

In [6]:
def shuffle(df: pd.DataFrame) -> pd.DataFrame:
    return df.reindex(np.random.permutation(df.index))

In [7]:
for split in ensemble.splits:
    ensemble.splits[split] = (shuffle(ensemble.splits[split][0])[:200000], ensemble.splits[split][1])

Tensorflow not installed


In [6]:
len(ensemble.test)

236498

In [8]:
splits = len(ensemble.splits)
scalers = [dmc.transformation.scale_raw_features] * splits
ignore = [None] * splits
ensemble.transform(scalers=scalers, ignore_features=ignore)

/usr/local/lib/python3.5/site-packages/scipy/sparse/compressed.py:739: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


In [9]:
res = []
for k in ensemble.splits:
    if len(ensemble.splits[k]['test'][1]) < 3000:
        continue
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.TheanoNeuralNetwork
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    res.append((k, prec, len(ye)))
    print(k, prec, len(ye), clf)

Tensorflow not installed
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6569043321299639 22160 <dmc.classifiers.TheanoNeuralNetwork object at 0x110f95198>
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup 0.661623958011732 3239 <dmc.classifiers.TheanoNeuralNetwork object at 0x110f95198>
unknown_articleID-known_customerID-known_voucherID-known_productGroup 0.6446157552397013 62265 <dmc.classifiers.TheanoNeuralNetwork object at 0x1126f1b70>
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6796371449033183 4189 <dmc.classifiers.TheanoNeuralNetwork object at 0x10e714860>
known_articleID-unknown_customerID-known_voucherID-known_productGroup 0.665740827187004 42844 <dmc.classifiers.TheanoNeuralNetwork object at 0x103962ef0>
known_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6425492610837438 6496 <dmc.classifiers.TheanoNeuralNetwork object at 0x114a27940>
known_articleID-known_customerID-known_vouch

In [11]:
resF = []
for k in ensemble.splits:
    if len(ensemble.splits[k]['test'][1]) < 3000:
        continue
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.Forest
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    resF.append((k, prec, len(ye)))
    print(k, prec, len(ye), clf)

Tensorflow not installed
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6670126353790613 22160 <dmc.classifiers.Forest object at 0x113c02630>
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6650200679221983 3239 <dmc.classifiers.Forest object at 0x1038cab70>
unknown_articleID-known_customerID-known_voucherID-known_productGroup 0.645193929173693 62265 <dmc.classifiers.Forest object at 0x111f36ba8>
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6691334447362138 4189 <dmc.classifiers.Forest object at 0x17b4e2c88>
known_articleID-unknown_customerID-known_voucherID-known_productGroup 0.662286434506582 42844 <dmc.classifiers.Forest object at 0x1038cadd8>
known_articleID-known_customerID-unknown_voucherID-known_productGroup 0.6427032019704433 6496 <dmc.classifiers.Forest object at 0x10e76bac8>
known_articleID-known_customerID-known_voucherID-known_productGroup 0.6462332478513257 93197 <dmc.classifiers.Forest objec

In [12]:
for k in ensemble.splits:
    Xt, yt = ensemble.splits[k]['train'][0], ensemble.splits[k]['train'][1]
    Xe, ye = ensemble.splits[k]['test'][0], ensemble.splits[k]['test'][1]
    fts, clf = ensemble.splits[k]['features'], dmc.classifiers.NaiveBayes
    clf = clf(Xt, yt)
    prec = dmc.evaluation.precision(ye, clf(Xe))
    print(k, prec, len(ye), clf)

unknown_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6056676272814601 2082 <dmc.classifiers.NaiveBayes object at 0x1038cabe0>
unknown_articleID-unknown_customerID-known_voucherID-unknown_productGroup 0.8571428571428572 14 <dmc.classifiers.NaiveBayes object at 0x1038cabe0>
unknown_articleID-unknown_customerID-known_voucherID-known_productGroup 0.6037003610108302 22160 <dmc.classifiers.NaiveBayes object at 0x1038cabe0>
unknown_articleID-known_customerID-unknown_voucherID-known_productGroup 0.5949367088607596 3239 <dmc.classifiers.NaiveBayes object at 0x1038cabe0>
unknown_articleID-known_customerID-known_voucherID-unknown_productGroup 0.6666666666666667 12 <dmc.classifiers.NaiveBayes object at 0x1038cabe0>
unknown_articleID-known_customerID-known_voucherID-known_productGroup 0.5875371396450655 62265 <dmc.classifiers.NaiveBayes object at 0x1038cabe0>
known_articleID-unknown_customerID-unknown_voucherID-known_productGroup 0.6144664597756028 4189 <dmc.classifiers.Naive

In [ ]:
Forest, Network, Bayes, SVM = dmc.classifiers.Forest, dmc.classifiers.TheanoNeuralNetwork, dmc.classifiers.Bayes, dmc.classifiers.SVM
raw_scaler, no_scaler = dmc.transformation.scale_raw_features, None
no_sample = None
params = {
    'uuuu': 
}